In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 9.0 MB/s 
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 47.6 MB/s 
     |████████████████████████████████| 596 kB 30.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn as nn
import pandas as pd

In [5]:
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

In [6]:
def getFinBERTVal(sentence) :
    inputs = tokenizer(
                        sentence,
                        return_tensors='pt',
                        truncation=True,
                        max_length=128,
                        padding='max_length',
                        add_special_tokens=True
    )

    output = model(input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
    )

    m = nn.Softmax(dim=1)
    result = m(output[0])

    negative_val = float(result[0][0])
    neutral_val = float(result[0][1])
    positive_val = float(result[0][2])

    return negative_val, neutral_val, positive_val

In [8]:
# from google.colab import files
# uploaded = files.upload()

Saving cleaned_report.xlsx to cleaned_report.xlsx


In [21]:
filename = "./combined_dataset_cleaned.xlsx"
df = pd.read_excel(filename)
# df = df.head(50)
df[['negative', 'neutral', 'positive', 'sentiment']] = pd.DataFrame([[0.0, 0.0, 0.0, ""]], index=df.index)

logCnt = 100
plusCnt = 100
for i in range(len(df)) :
    if(i == logCnt) :
        print(i)
        logCnt += plusCnt
        
    negVal, neuVal, posVal = getFinBERTVal(df['title'][i])
    sentDict = {"Negative" : negVal, "Neutral" : neuVal, "Positive" : posVal}
    maxVal = max(sentDict, key=sentDict.get)
    df['negative'][i] = negVal
    df['neutral'][i] = neuVal
    df['positive'][i] = posVal
    df['sentiment'][i] = maxVal
print(df)

C:\Users\user\AppData\Local\Temp/ipykernel_9864/194779996.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['negative'][i] = negVal
C:\Users\user\AppData\Local\Temp/ipykernel_9864/194779996.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['neutral'][i] = neuVal
C:\Users\user\AppData\Local\Temp/ipykernel_9864/194779996.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['positive'][i] = posVal
C:\Users\user\AppData\Loca

         date                                              title  negative  \
0  2021-06-03                        코스피, 5거래일 연속 상승…최고치 경신 '코앞'  0.000369   
1  2021-06-03         결국 ‘4번 타자’는 삼성전자?… 삼전 급반등, 코스피 다시 3250선 눈앞  0.027041   
2  2021-06-07              예금 대신 주식·ETF로 자산 채워···MZ세대 '재테크 계좌' 찜  0.000040   
3  2021-06-08                      동학개미운동 성공 이후… 삼성전자 주가 주무르는 외인  0.063891   
4  2021-06-08                    삼성전자부터 LG화학까지...국민주 목표주가 줄줄이 하향  0.999561   
5  2021-06-08                "주변에 돈 번 사람 없다"…주가 치솟는데 개미 수익률은 역주행  0.010620   
6  2021-06-10  삼성전자, 8나노 RF 공정 기술 개발 및 0.64㎛ 이미지센서 출시 이어지는 호재...  0.000051   
7  2021-06-11                    LG유플·에스오일·삼전…중간배당 시즌 '여름보너스' 기대  0.002938   
8  2021-06-13                삼전 팔아 2억 차익, 세금도 안낸다? 주식 양도세 '절세 팁'  0.000073   
9  2021-06-13                      윤지호 이베스트 리서치센터장 "삼성전자만 답 아니다"  0.008788   
10 2021-06-19                   "이제 때가 왔다, 삼전 대신 이 종목 사라"…전문가 조언  0.000095   
11 2021-06-21               삼성전자 '7만전자'로 후퇴…외국인·기관 '팔고' 개인은 '사고'

In [22]:
print("Done!")

Done!


In [23]:
df.to_excel("./combined_dataset_cleaned_finBERT.xlsx", index = False)